In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os, sys, json, joblib
import pandas as pd
from sklearn.metrics import r2_score

In [ ]:
files = sorted(glob('./*/*/*results.json'))

In [ ]:

quantum_results = []
quantum_data = []
for i in files:
    print(i)
    with open(i,'r') as f:
        data = json.load(f)
    ratio = os.path.basename(os.path.dirname(os.path.dirname(i)))
    quantum_data.append((ratio,data['R2_train'],data['R2_test']))
    quantum_results.append(('A2_HWE-CNOT',ratio,'R2_train',data['R2_train']))
    quantum_results.append(("A2_HWE-CNOT",ratio,'R2_test',data['R2_test']))

quantum_data = np.array(quantum_data).astype(float)

In [ ]:

trainsize=[ 0.1,0.3, 0.5, 0.7,0.8]
classical_models=['ridge', 'lasso', 'elastic', 'knn', 'rfr', 'grad', 'svr', 'krr', 'gpr'
]

classical_results=[]

for models in classical_models:
    for ts in trainsize:
        
        with open(f"./classical/{ts}_{models}_scores.json",'r') as f:
            classical_dict=json.load(f)
        classical_results.append((models,ts,'R2_train',classical_dict['R2_train']))
        classical_results.append((models,ts,'R2_test',classical_dict['R2_test']))
            

In [ ]:
classical_df=pd.DataFrame(classical_results,columns=['model','ratio','set','value'])

In [ ]:
quantum_df=pd.DataFrame(quantum_results,columns=['model','ratio','set','value'])

In [ ]:
combined_df = pd.concat([classical_df,quantum_df])

In [ ]:
combined_df.query("model == 'A2_HWE-CNOT' & ratio == 0.8").round(4)

In [ ]:
# combined_df.pivot(columns=['set'],values='value',index=['ratio','model']).to_excel(os.path.join(os.path.expanduser('~'),'qregress/spreadsheets/DDCC5_lc.xlsx'))

print(combined_df.pivot(columns=['set'],values='value',index=['ratio','model']).sort_index(axis=1,ascending=False).round(4).to_latex(float_format="{:.4f}".format))

In [ ]:
for ts in trainsize:
    print(ts)
    for m in classical_models:
        cdf=classical_df[(classical_df['ratio']==ts)&(classical_df['model']==m)].set_index('set')
        qdf=quantum_df[quantum_df['ratio']==f'{ts}'].set_index('set')
        print(m,all(cdf['value']>=qdf['value']))
        if cdf['value'].loc['R2_test']>=qdf['value'].loc['R2_test']:
            print(cdf['value'].loc['R2_test'],qdf['value'].loc['R2_test'])
    print()

In [ ]:
combined_df = combined_df.astype({'model':str,'ratio':float,"set":str,'value':float})
combined_df['set'] = [i.split('_')[1].capitalize() for i in combined_df['set']]

train = combined_df[(combined_df['model']=='A2_HWE-CNOT')&(combined_df['set']=='Train')]
test = combined_df[(combined_df['model']=='A2_HWE-CNOT')&(combined_df['set']=='Test')]

drop_pqc = combined_df[combined_df['model']!='A2_HWE-CNOT']

In [ ]:
combined_df

In [ ]:
drop_pqc.min()

In [ ]:


fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,5),sharey=True)

sns.boxplot(data=drop_pqc,x='ratio',y='value',hue='set', native_scale=True,ax=ax1,palette=sns.color_palette('Paired',2))

# ins = ax1.inset_axes([0.55,0.15,0.4,0.4])
# sns.boxplot(data=drop_pqc[drop_pqc['ratio']==0.1],x='ratio',y='value',hue='set', native_scale=True,palette=sns.color_palette('Paired',2),ax=ins)
# ins.set_ylim(-0.5,1.05)
# ins.set_ylabel('R$^{2}$')
# ins.set_xlabel('Training Set Ratio')
# ins.set_xticks([0.1])
# ins.get_legend().remove()

ax1.set_ylim(0,1.01)
ax1.set_xlim(0,0.9)
ax1.set_ylabel('R$^{2}$')
ax1.set_xlabel('Training Set (%)')
ax1.set_title('Classical Models')
# Convert x-axis labels to percentages
xticks = np.linspace(0, 0.9, 10)
ax1.set_xticks(xticks)
ax1.set_xticklabels([f"{int(x * 100)}%" for x in xticks])

ax2.plot(train['ratio'],train['value'],'-x',color=sns.color_palette('Paired',2)[0],label='Train')
ax2.plot(test['ratio'],test['value'],'-o',color=sns.color_palette('Paired',2)[1],label='Test')
ax2.legend()
ax2.set_xticks(xticks)
ax2.set_xticklabels([f"{int(x * 100)}%" for x in xticks])

# ax2.set_ylim(0.95,1)
ax2.set_xlim(0.0,0.9)
ax2.set_ylabel('R$^{2}$')
ax2.set_xlabel('Training Set (%)')
ax2.set_title('A2_HWE-CNOT')
plt.tight_layout()
plt.savefig(f"{os.path.expanduser('~')}/qregress/images/DDCC/DDCC_learning_curves.png", bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:
combined_df.query("model == 'A2_HWE-CNOT' & set == 'Train' ").round(4)

In [ ]:
combined_df.query("model == 'A2_HWE-CNOT' & set == 'Test' ").round(4)

In [ ]:
drop_pqc_adj = drop_pqc.copy()

In [ ]:
r_df = pd.DataFrame(quantum_data,columns=['index','Train','Test']).melt(id_vars=['index'], value_vars=['Train','Test'])
r_df['variable'] = [i+" (A2_HWE-CNOT)" for i in r_df['variable']]

drop_pqc_adj['set']=[i+" (classical)" for i in drop_pqc['set']]
sns.lineplot(data=drop_pqc_adj,x='ratio',y='value',hue='set',style='set',palette=sns.color_palette('Paired',4)[2:])

ax = sns.lineplot(r_df,x='index',y='value',hue='variable',style='variable',palette=sns.color_palette('Paired',2),markers=['>','<'])
plt.ylim(0,1.05)
# plt.gca().xaxis.set_major_formatter(mticker.PercentFormatter(xmax=100))  # Adjust xmax if needed
# ax.set_xtick([f"{int(float(i)*100)}%" for i in np.array(r_df['index'],dtype=float)])
ax.set_xticks(xticks)
ax.set_xticklabels([f"{int(x * 100)}%" for x in xticks])

plt.ylabel('R$^{2}$')
plt.xlabel('Training Set (%)')
plt.title('A2_HWE-CNOT')
plt.legend(loc=4)
plt.tight_layout()
plt.savefig(f"{os.path.expanduser('~')}/qregress/images/DDCC/DDCC_learning_curves.png", bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
drop_pqc_adj = drop_pqc.copy()
# Assuming drop_pqc_adj and quantum_data are already defined
r_df = pd.DataFrame(quantum_data, columns=['index', 'Train', 'Test']).melt(id_vars=['index'], value_vars=['Train', 'Test'])
r_df['variable'] = [i + " (A2_HWE-CNOT)" for i in r_df['variable']]

drop_pqc_adj['set'] = [i + " (classical)" for i in drop_pqc_adj['set']]

# Main plot
# ax1 = sns.lineplot(data=drop_pqc_adj, x='ratio', y='value', hue='set', style='set', palette=(sns.color_palette('Paired', 6)[3],sns.color_palette('Paired', 6)[5]),errorbar=lambda x: (x.min(), x.max()),err_style='bars')
ax = sns.lineplot(data=r_df, x='index', y='value', hue='variable', style='variable', palette=sns.color_palette('Paired', 2), markers=['>', '<'])
plt.ylim(0, 1.05)
ax.set_xticks(xticks)
ax.set_xticklabels([f"{int(x * 100)}%" for x in xticks])
plt.ylabel('R$^{2}$')
plt.xlabel('Training Set (%)')
plt.title('A2_HWE-CNOT')
plt.legend(loc=3)

# Inlay plot
inset_ax = inset_axes(ax, width="40%", height="40%", loc=4, bbox_to_anchor=(0, 100, 600, 400))  # Adjust location as needed

# Plot with legend temporarily enabled
sns_plot = sns.lineplot(
    data=drop_pqc_adj,
    x='ratio',
    y='value',
    hue='set',
    style='set',
    palette=(sns.color_palette('Paired', 6)[3], sns.color_palette('Paired', 6)[5]),
    errorbar=lambda x: (x.min(), x.max()),
    ax=inset_ax,
    err_style='bars',
    legend=True  # Enable legend just to capture handles
)

# Get handles and labels
handles, labels = inset_ax.get_legend_handles_labels()

# Manually reset legend with new labels
inset_ax.legend(loc=4,handles=handles, labels=["Train", "Test"], fontsize='small',ncols=2)

# Final axis formatting
inset_ax.set_ylim(0.98, 1.001)
inset_ax.set_xticks(xticks[0::2])
inset_ax.set_xticklabels([f"{int(x * 100)}%" for x in xticks[0::2]])
inset_ax.set_ylabel('R$^{2}$')
inset_ax.set_xlabel('Training Set (%)')
inset_ax.set_title('Classical Models')
plt.tight_layout()
plt.savefig(f"{os.path.expanduser('~')}/qregress/images/DDCC/DDCC_learning_curves.png", bbox_inches='tight', pad_inches=0.1)
plt.show()


In [ ]:
combined_df